## Paralelización

In [1]:
import random
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
import yfinance as yf
from matplotlib import cm
import scipy.optimize as opt
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from scipy.optimize import minimize
from joblib import Parallel, delayed 
import matplotlib.colorbar as colorbar
from matplotlib.colors import Normalize
from IPython.display import display, Markdown
from matplotlib.colors import LinearSegmentedColormap

### Simulación de portafolios

In [3]:
tickers = ['AAPL', 'NVDA', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'SPOT', 'SBUX', 'JPM', 'AXP', 'MCD', 'KO', 
           'NFLX', 'CMG', 'CP', 'WMT', 'V', 'GLD', 'BLK', 'PG', 'JNJ', 'TMUS', 'MA', 'BX', 'LULU', 'DPZ', 'BAC', 
           'FDX', 'DIS', 'GE', 'HSY', 'HP', 'COST', 'HD', 'K', 'ADBE', 'CSCO', 'T', 'F', 'NKE', 'CVX', 'XOM', 'PYPL', 
           'PEP', 'PFE', 'MRNA', 'RL', 'AZN', 'BABA', 'VZ', 'WBD', 'HSBC', 'UBER']